In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from matplotlib import style

from pathlib import Path
from utils import read_data

style.use("seaborn-v0_8")

RECOMPUTE = True

project_dir = Path(".")
data_dir = project_dir / "data"
results_dir = project_dir / "results"

data_slo = read_data(data_dir=data_dir, cohort="slo", version="final", recompute=RECOMPUTE)
data_por = read_data(data_dir=data_dir, cohort="por", version="final", recompute=RECOMPUTE)

data_raw = pd.concat([data_slo, data_por], axis=0)
data_raw.sample(3, random_state=3)

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from utils import X_COLUMN_ORDER
from utils import COLUMN_ORDER
from utils import Y_COLUMN
from utils import BINARY_CATEGORICAL_COLUMNS
from utils import MULTI_CATEGORICAL_COLUMNS
from utils import CLASS_NAMES
from utils import impute_and_scale_data

In [ ]:
data, scaling_info = impute_and_scale_data(data_raw, mask_predicate=lambda row: row["cohort"] == "slo")

In [ ]:
from utils import train_model_and_cv
from utils import X_COLUMNS
from sklearn.linear_model import LogisticRegression

BASE_MODEL = LogisticRegression(random_state=0, penalty="l2", max_iter=100)
PARAM_GRID = {
    "class_weight": ["balanced", None],
    "C": [1 / 128, 1 / 64, 1 / 32, 1 / 16, 1 / 8, 1 / 4, 1.0, 4.0, 16.0, 64.0, 128.0],
    "fit_intercept": [True, False],
}
RANDOM_STATE = 3
SIZE_TEST_SPLIT = 0.4

data["split"] = "test"
indices_train_val, _ = train_test_split(
    data[data["cohort"] == "slo"].index,
    test_size=SIZE_TEST_SPLIT,
    random_state=RANDOM_STATE,
    stratify=data[data["cohort"] == "slo"][Y_COLUMN],
)
data.loc[indices_train_val, "split"] = "train_val"

model, df_cv = train_model_and_cv(
    model=BASE_MODEL,
    param_grid=PARAM_GRID,
    X=data[data["split"] == "train_val"][X_COLUMNS],
    y=data[data["split"] == "train_val"][Y_COLUMN],
    cv=3,
    scoring="roc_auc",
)

In [ ]:
from sklearn.metrics import roc_auc_score, classification_report
import numpy as np
from utils import filter_by_metadata

for cohort, version, split in [
    ("slo", "final", "train_val"),
    ("slo", "final", "test"),
    ("por", "final", "test"),
]:
    print(f"Split: {split}, '{cohort}', {version}")
    data_subset = filter_by_metadata(data, cohort=cohort, version=version, split=split)
    
    y_true = data_subset[Y_COLUMN]
    y_pred = model.predict(data_subset[X_COLUMNS])
    report = classification_report(y_true, y_pred, target_names=CLASS_NAMES)
    print(report)

    print("AUC:", roc_auc_score(
        y_true=data_subset[Y_COLUMN],
        y_score=model.predict_proba(data_subset[X_COLUMNS])[:, 1],
    ))
    print("\n")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from utils import compute_metrics

# Sensitivity = TP Rate = Recall = TP / P
# Specificity = TN Rate = TN / N

metrics = []
for t in np.linspace(0.01, 1.00, 100):
    recall_pos_slo, recall_neg_slo, precision_pos_slo = compute_metrics(
        filter_by_metadata(data, cohort="slo", version="final", split="test"),
        model=model,
        threshold=t
    )

    recall_pos_por, recall_neg_por, precision_pos_por = compute_metrics(
        filter_by_metadata(data, cohort="por", version="final", split="test"),
        model=model,
        threshold=t
    )

    metrics.append({
        "threshold": t,
        # Slovenia
        "specificity (slo/test)": recall_neg_slo,
        "sensitivity/recall (slo/test)": recall_pos_slo,
        "precision (slo/test)": precision_pos_slo,
        # Portugal
        "specificity (por/test)": recall_neg_por,
        "sensitivity/recall (por/test)": recall_pos_por,
        "precision (por/test)": precision_pos_por,
    })

metrics_df = pd.DataFrame(metrics)

import matplotlib.pyplot as plt
from matplotlib import style

fig, ax = plt.subplots(figsize=(8, 6))
ax.set_title("Specificity-Sensitivity Curve")
ax.set_xlabel("Sensivity")
ax.set_ylabel("Specificity")
ax.plot(
    metrics_df["sensitivity/recall (slo/test)"],
    metrics_df["specificity (slo/test)"],
    color="tab:green",
    label="SLO (Test split)",
)
ax.plot(
    metrics_df["sensitivity/recall (por/test)"],
    metrics_df["specificity (por/test)"],
    color="tab:red",
    label="POR (Test split)",
)
plt.legend(loc="lower left")
metrics_df.to_excel(results_dir / "specificity_sensitivity.xlsx")

metrics_df[metrics_df["specificity (slo/test)"] > 0.949]

In [ ]:
assert model.coef_.shape == (1, len(model.feature_names_in_))

pd.DataFrame({
    "feature_name": model.feature_names_in_,
    "weight": model.coef_[0],
    
}).sort_values("weight", ascending=False)

In [ ]:
assert (data_raw.index == data.index).all()

data_raw["split"] = data["split"]
data_raw["predicted_probability"] = pd.Series(model.predict_proba(data[X_COLUMNS])[:, 1], index=data.index)
data_raw.to_excel(results_dir / "model_split_probability.xlsx")
data_raw.head(3)

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(12, 8))
ax.set_title("Precision-Recall Curve")
for cohort, version, split, kwargs in [
    ("slo", "final", "train_val", {"color": "tab:green", "alpha": 0.5}),
    ("slo", "final", "test", {"color": "tab:green", "alpha": 1.0}),
    ("por", "final", "test", {"color": "tab:red", "alpha": 0.5}),    
    # ("slo", 2.0, "train_val", {"color": "tab:green", "alpha": 0.5}),
    # ("slo", 2.0, "test", {"color": "tab:green", "alpha": 1.0}),
    # ("por", 2.0, "test", {"color": "tab:red", "alpha": 0.5}),
    # ("por", 3.0, "test", {"color": "tab:red", "alpha": 1.0}),
]:    
    print(f"Split: {split}, {cohort}/{version}")
    data_subset = filter_by_metadata(data, cohort=cohort, version=version, split=split)
    
    y_true = data_subset[Y_COLUMN]
    y_pred = model.predict_proba(data_subset[X_COLUMNS])[:, 1]

    display = PrecisionRecallDisplay.from_predictions(
        y_true=y_true,
        y_pred=y_pred,
        name=f"{split}, {cohort}/{version}",
        plot_chance_level=False,
        drop_intermediate=True,
        ax=ax,
        **kwargs
    )

In [ ]:
from inference import model_fn
from inference import preprocess_sample

def check_dicts_close(a: dict, b: dict, tol: float = 1e-4) -> None:
    assert set(a) == set(b)
    for key in a:
        assert type(a[key]) is type(b[key]), (key, a[key], b[key])
        if isinstance(a[key], float):
            assert abs(a[key] - b[key]) < tol, (key, a[key], b[key])
        else:
            assert a[key] == b[key], (key, a[key], b[key])


debug = False

for sample_id in data_raw.index:
    sample_raw = json.loads(data_raw.loc[sample_id, X_COLUMN_ORDER].to_json())

    sample = preprocess_sample(sample_raw, debug=debug)
    expected_sample = data.loc[sample_id, X_COLUMNS].to_dict()
    for feature_name_bin in BINARY_CATEGORICAL_COLUMNS:
        expected_sample[feature_name_bin] = float(expected_sample[feature_name_bin])
    
    if debug:
        print(json.dumps(expected_sample, indent=4))

    check_dicts_close(sample, expected_sample)

    probability = model_fn(sample)
    expected_probability = float(model.predict_proba(data.loc[[sample_id], X_COLUMNS])[0, 1])
    
    assert abs(probability - expected_probability) < 1e-04, (probability, expected_probability)    

In [ ]:
import math
import json

WEIGHTS = {
    feature_name: float(weight)
    for feature_name, weight in zip(
        model.feature_names_in_, model.coef_[0]
    )
}
INTERCEPT = float(model.intercept_[0])

print(INTERCEPT)
print(json.dumps(WEIGHTS, indent=4))
print(json.dumps(scaling_info, indent=4))

BINARY_CATEGORICAL_COLUMNS = [
    "gender",  # 2
    "fh_xant",  # 2
    "fh_acrus_senilis",  # 2
    # "gen_conf_fh",  # 2
]
MULTI_CATEGORICAL_COLUMNS = [
    "fh_high_cholesterol",  # 4
    "fh_premature_cad",  # 4
    "fh_pad_cvi",  # 4
]


INTERCEPT = -8.330929160368113
WEIGHTS = {
    "age": 0.00667784927430266,
    "gender": 0.32143804346854776,
    "fh_high_cholesterol_1": 0.34873544396742856,
    "fh_high_cholesterol_2": 0.02078297866340733,
    "fh_high_cholesterol_3": 0.643902650173691,
    "fh_premature_cad_1": 0.05226457867177578,
    "fh_premature_cad_2": 0.12812175064921372,
    "fh_premature_cad_3": 0.03427669157864839,
    "fh_pad_cvi_1": 0.02237060945544122,
    "fh_pad_cvi_2": -0.364993539338868,
    "fh_pad_cvi_3": 0.00978393164164006,
    "fh_xant": -0.21087954347939916,
    "fh_acrus_senilis": 0.2413157398789852,
    "hdl_cholesterol": -0.8760712481075249,
    "ldl_cholesterol": 1.3552895085240824,
    "total_cholesterol": 1.1470281822853394,
    "tag": -0.5361077381018685,
    "bmi_z_score": -0.05708627050403221,
    "lp_a": -0.4028915834363285
}
SCALING_INFO = {
    "age": {
        "mean": 7.314633123689728,
        "std": 2.607213078684607
    },
    "hdl_cholesterol": {
        "mean": 1.5362264150943397,
        "std": 0.37043406815321883
    },
    "ldl_cholesterol": {
        "mean": 3.78845283018868,
        "std": 1.1593288141513893
    },
    "total_cholesterol": {
        "mean": 5.767471698113208,
        "std": 1.1603039079279096
    },
    "tag": {
        "mean": 1.0961132075471698,
        "std": 0.7718188209096246
    },
    "bmi_z_score": {
        "mean": 0.28694020169346707,
        "std": 1.3078173489609493
    },
    "lp_a": {
        "mean": 310.6692307692308,
        "std": 332.1171687025873
    }
}



def preprocess_sample(raw_sample: dict) -> dict:
    skip_columns = {"cohort", "predicted_probability", "version", "gen_conf_fh", "split"}
    assert set(raw_sample) - skip_columns == set(X_COLUMN_ORDER)

    print(json.dumps(raw_sample, indent=4))
    
    sample = {}
    for feature_name, feature_value in raw_sample.items():
        if feature_name in skip_columns:
            continue
        
        if feature_name in BINARY_CATEGORICAL_COLUMNS:
            sample[feature_name] = float(feature_value)
        elif feature_name in MULTI_CATEGORICAL_COLUMNS:
            assert isinstance(feature_value, int)
            for value in [1, 2, 3]:
                sample[f"{feature_name}_{value}"] = 1.0 if feature_value == value else 0.0
        elif feature_name in scaling_info:
            assert isinstance(feature_value, float)
            mean = scaling_info[feature_name]["mean"]
            std = scaling_info[feature_name]["std"]
            sample[feature_name] = float((feature_value - mean) / std )
        else:
            assert False, feature_name

    print(json.dumps(sample, indent=4))

    assert len(set(sample) - set(X_COLUMNS)) == 0, set(sample) - set(X_COLUMNS)
    assert len(set(X_COLUMNS) - set(sample)) == 0, set(X_COLUMNS) - set(sample)
        
    return sample

def model_fn(sample: dict) -> float:
    assert set(X_COLUMNS) == set(sample)
    assert set(WEIGHTS) == set(sample)

    def _sigmoid(x: float) -> float:
        return 1 / (1 + math.exp(-x))

    # Logistic regression
    weighted_sum = INTERCEPT
    for feature_name, weight_value in WEIGHTS.items():
        feature_value = sample[feature_name]
        weighted_sum += weight_value * feature_value

    return _sigmoid(weighted_sum)
    
        
    

raw_sample = data_raw.iloc[0].to_dict()
model_fn(preprocess_sample(raw_sample))